In [ ]:
from pypower.api import loadcase, runopf, ppoption
from pypower.idx_brch import *
from pypower.idx_bus import *
from pypower.idx_gen import *
from pypower.idx_cost import *
import oct2py
import numpy as np
import math
import os
from tqdm import tqdm
from oct2py import Oct2Py
octave = Oct2Py()
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import networkx as nx
import pickle
import shutil
import cvxpy as cp
if shutil.which('latex') is None:
    LATEX_AVAILABLE = False
else:
    LATEX_AVAILABLE = True
if LATEX_AVAILABLE:
    plt.rcParams['font.size'] = 14
    plt.rcParams['text.usetex'] = True
    plt.rcParams['text.latex.preamble'] = r'\usepackage{amsmath,amssymb}'
    
MAX_BUS = 1000 # upper limit of number of buses in cases to be considered

In [ ]:
# get all cases in current directory
current_directory = os.getcwd()
all_files_and_directories = os.listdir(current_directory)
case_files = [os.path.join(current_directory, f) for f in all_files_and_directories if f.endswith('.m') and os.path.isfile(os.path.join(current_directory, f))]

cases, casenames = [], []
cases_full, casenames_full = [], []
for cf in (t:=tqdm(case_files)):
    octave.source(os.path.basename(cf))
    cname = os.path.basename(cf).split('.')[0]
    num_buses = None
    for ci in cname.split('_'):
        if 'case' in ci:
            num_buses = int(''.join([chr for chr in ci.replace('case','',1) if chr.isdigit()]))
    if num_buses <= MAX_BUS:
        cases.append(loadcase(octave.feval(cname)))
        casenames.append(cname)
    cases_full.append(loadcase(octave.feval(cname)))
    casenames_full.append(cname)
    t.set_description(f"Loaded {os.path.basename(cf)}")

In [ ]:
# this_case = cases[0]

for cn,this_case in tqdm(zip(casenames,cases)):
    to_str = lambda x: [str(int(xi)) for xi in x]
    bus_list = to_str(this_case['bus'][:,BUS_I].tolist())
    # branches
    BR_ON = this_case['branch'][:,BR_STATUS].astype(bool).astype(int)
    from_bus = to_str(this_case['branch'][:,F_BUS][BR_ON].tolist())
    to_bus = to_str(this_case['branch'][:,T_BUS][BR_ON].tolist())
    br_r = this_case['branch'][:,BR_R][BR_ON].tolist()
    br_x = this_case['branch'][:,BR_X][BR_ON].tolist()
    br_b = this_case['branch'][:,BR_B][BR_ON].tolist()
    tap = this_case['branch'][:,TAP][BR_ON].tolist()
    shift = this_case['branch'][:,SHIFT][BR_ON].tolist()
    branch_list = [(f,t) for f,t in zip(from_bus,to_bus)]
    # generators
    GEN_ON = this_case['gen'][:,GEN_STATUS].astype(bool).astype(int)
    gen_to_bus = this_case['gen'][:,GEN_BUS][GEN_ON]
    gen_mbase = this_case['gen'][:,MBASE][GEN_ON]
    gen_pmax, gen_pmin = this_case['gen'][:,PMAX][GEN_ON], this_case['gen'][:,PMIN][GEN_ON]
    gen_qmax, gen_qmin = this_case['gen'][:,QMAX][GEN_ON], this_case['gen'][:,QMIN][GEN_ON]
    # generator cost model
    gmodel = this_case['gencost'][:,MODEL]
    elements, counts = np.unique(gmodel, return_counts=True)
    most_frequent = elements[counts.argmax()]
    anyother = bool(np.any(elements!=most_frequent))
    print(f"For case {cn}, the most frequent model type is {most_frequent}, presence of any other model is {anyother}, coeffs size is {this_case['gencost'][:,COST:].shape[1]}.")

In [ ]:
for this_case,cn in tqdm(zip(cases,casenames)):
    to_str = lambda x: [str(int(xi)) for xi in x]
    bus_list = to_str(this_case['bus'][:,BUS_I].tolist())
    bus_list_raw = this_case['bus'][:,BUS_I].astype(int)
    bus_basekv = this_case['bus'][:,BASE_KV]
    bus_vmax, bus_vmin = this_case['bus'][:,VMAX], this_case['bus'][:,VMIN]
    bus_pd, bus_qd = this_case['bus'][:,PD], this_case['bus'][:,QD]
    bus_gs, bus_bs = this_case['bus'][:,GS] / this_case['baseMVA'], this_case['bus'][:,BS] / this_case['baseMVA']
    # branches
    BR_ON = this_case['branch'][:,BR_STATUS].astype(bool)
    from_bus = to_str(this_case['branch'][:,F_BUS][BR_ON].tolist())
    to_bus = to_str(this_case['branch'][:,T_BUS][BR_ON].tolist())
    br_r = this_case['branch'][:,BR_R][BR_ON]
    br_x = this_case['branch'][:,BR_X][BR_ON]
    br_b = this_case['branch'][:,BR_B][BR_ON]
    tap = np.where(abs(this_case['branch'][:,TAP][BR_ON]) <1e-4, 1., this_case['branch'][:,TAP][BR_ON])
    shift = this_case['branch'][:,SHIFT][BR_ON]
    angmin,angmax = np.where(np.isclose(this_case['branch'][:,ANGMIN],0),-360*np.ones_like(this_case['branch'][:,ANGMIN]),this_case['branch'][:,ANGMIN]), np.where(np.isclose(this_case['branch'][:,ANGMAX],0),360*np.ones_like(this_case['branch'][:,ANGMAX]),this_case['branch'][:,ANGMAX])
    flow_lim = (this_case['branch'][:,RATE_A][BR_ON])
    branch_list = [(f,t) for f,t in zip(from_bus,to_bus)]
    # generators
    GEN_ON = this_case['gen'][:,GEN_STATUS].astype(bool)
    gen_to_bus = this_case['gen'][:,GEN_BUS][GEN_ON]
    gen_mbase = this_case['gen'][:,MBASE][GEN_ON]
    gen_pmax, gen_pmin = this_case['gen'][:,PMAX][GEN_ON], this_case['gen'][:,PMIN][GEN_ON]
    gen_qmax, gen_qmin = this_case['gen'][:,QMAX][GEN_ON], this_case['gen'][:,QMIN][GEN_ON]
    gencost = this_case['gencost'][:,COST:][GEN_ON]
    # generator cost model
    gmodel = this_case['gencost'][:,MODEL][GEN_ON]
    elements, counts = np.unique(gmodel, return_counts=True)
    most_frequent = elements[counts.argmax()]
    anyother = bool(np.any(elements!=most_frequent))
    print(f"For case {cn}, the most frequent model type is {most_frequent}, presence of any other model is {anyother}, coeffs size is {this_case['gencost'][:,COST:].shape[1]}.")
    # create directed networkx graph
    G = nx.DiGraph()
    for br,r,x,b,t,s,f,amin,amax in zip(branch_list,br_r,br_x,br_b,tap,shift,flow_lim,angmin,angmax):
        # impedances
        yl = 1. / (r + 1j*x)
        tl = t*(math.cos(math.radians(s)) + 1j * math.sin(math.radians(s)))
        yff,ytt = (yl + 1j * (b/2)) / (tl*tl.conjugate()) , (yl + 1j * (b/2))
        yft,ytf = - yl / tl.conjugate(), -yl / tl
        # pu for normalizing voltages
        brf,brt = int(br[0]),int(br[1])
        kv_f,kv_t = bus_basekv[bus_list.index(br[0])], bus_basekv[bus_list.index(br[1])]
        G.add_edge(u_of_edge=br[0],v_of_edge=br[1],yff=yff,yft=yft,ytf=ytf,ytt=ytt,pu_fac=(kv_f*kv_t).item(),kv_f=kv_f.item(),kv_t=kv_t.item(),kvfsq=(kv_f*kv_f).item(),fmax=f,angmin=amin,angmax=amax,tap=t,shift=s)
    # populate nodes with relevant data
    for bus,kv,pd,qd,vmin,vmax,gs,bs in zip(bus_list_raw,bus_basekv,bus_pd,bus_qd,bus_vmin,bus_vmax,bus_gs,bus_bs):
        busidx = bus_list_raw.tolist().index(bus)
        cgen = np.where(gen_to_bus==bus)[0] if np.where(gen_to_bus==bus)[0].size!=0 else None
        kwlist = {'kvsq':kv*kv,'pd':pd,'qd':qd,'vmin':vmin,'vmax':vmax,'ys':gs+1j*bs,'conn_gen_idx':cgen,'basemva':this_case['baseMVA'],'basegen':gen_mbase,'basekv':bus_basekv[np.where(bus_list_raw==bus)]}
        attr = {str(int(bus)):kwlist}
        nx.set_node_attributes(G, attr)
    # define constants
    n_bus = G.number_of_nodes()
    n_branch = G.number_of_edges()
    n_gen = gen_to_bus.size
    # cp variables
    Sf = cp.Variable(n_branch,complex=True) # p.u.
    St = cp.Variable(n_branch,complex=True) # p.u.
    Sgen = cp.Variable(n_gen,complex=True) # p.u.
    U = cp.Variable(n_bus,nonneg=True) # p.u.
    W = cp.Variable(n_branch,complex=True) # p.u.
    # constraint container
    constr = []
    
    # indicators00                                  
    DISABLE_MODEL = False
    DISABLE_FLOWLIMS = False
    DISABLE_ANG = False
    DISABLE_SOCR = False
    DISABLE_BALANCE = False
    DISABLE_VLIMS = False
    DISABLE_GENLIMS = False
    # branch constraints
    edge_record = []
    for bidx,bdata in enumerate(G.edges(data=True)):
        idx_from,idx_to = bus_list.index(bdata[0]), bus_list.index(bdata[1])
        edge_record.append((bidx,bdata[0],bdata[1]))
        bdata = bdata[2] 
        # branch flow (p.u.)
        if not DISABLE_MODEL:
            constr.append(Sf[bidx] == bdata['yff'].conjugate()*U[idx_from] + bdata['yft'].conjugate()*W[bidx])
            constr.append(St[bidx] == bdata['ytt'].conjugate()*U[idx_to] + bdata['ytf'].conjugate()*cp.conj(W[bidx]))
        # branch flow limits (p.u.)
        if not DISABLE_FLOWLIMS:
            constr.append(cp.abs(Sf) <= bdata['fmax'] / this_case['baseMVA'])
            constr.append(cp.abs(St) <= bdata['fmax'] / this_case['baseMVA'])
        # # branch angle limits (unitless)
        if not DISABLE_ANG:
            constr.append(math.tan(math.radians(bdata['angmin']))*cp.real(W[bidx]) <= cp.imag(W[bidx]))
            constr.append(math.tan(math.radians(bdata['angmax']))*cp.real(W[bidx]) >= cp.imag(W[bidx]))
        # conic constraints (SOC relaxation) (p.u.)
        if not DISABLE_SOCR:
            constr.append(cp.norm(cp.hstack([U[idx_from]-U[idx_to],2*cp.real(W[bidx]),2*cp.imag(W[bidx])]),2) <= U[idx_from]+U[idx_to])
    # bus constraints
    attr1, attr2 = [], []
    for nidx,ndata in enumerate(G.nodes(data=True)):
        ndata = ndata[1]
        cur_bus = bus_list[nidx]
        in_edges,out_edges = [],[]
        # record in and out edges
        for busname in bus_list:
            for itm in edge_record:
                if itm[1]==cur_bus and itm[2]==busname:
                    out_edges.append(itm[0])
                if itm[2]==cur_bus and itm[1]==busname:
                    in_edges.append(itm[0])
        # nodal balance (standard units)
        Sgen_total = cp.sum([Sgen[gidx] for gidx in ndata['conn_gen_idx']]) if ndata['conn_gen_idx'] is not None else 0
        Sd = (ndata['pd'] + 1j*ndata['qd']) / this_case['baseMVA']
        S_out = cp.sum([St[bidx] for bidx in out_edges]) if out_edges is not [] else 0 # outward flow
        S_in = cp.sum([Sf[bidx] for bidx in in_edges]) if in_edges is not [] else 0 # inbound flow
        if not DISABLE_BALANCE:
            constr.append(Sgen_total-Sd
                        -ndata['ys'].conjugate()*U[nidx]
                        ==S_in+S_out)
                        #-P_in-P_out==0)
        # else:
        #     expr1P, expr2P, expr3P, expr4P, expr5P = Pgen_total,-Pd,-ndata['ys'].real*U[nidx],P_in+P_out,ndata['basekv']
        #     expr1Q, expr2Q, expr3Q, expr4Q, expr5Q = Qgen_total,-Qd,-ndata['ys'].imag*U[nidx],Q_in+Q_out,ndata['basekv']
        #     attr1.append((expr1P,expr2P,expr3P,expr4P,expr5P))
        #     attr2.append((expr1Q,expr2Q,expr3Q,expr4Q,expr5Q))
        # voltage limits (no angle limits here since current model only computes angle differences) (p.u.)
        if not DISABLE_VLIMS:
            constr.append(U[nidx]>=np.square(ndata['vmin']))
            constr.append(U[nidx]<=np.square(ndata['vmax']))
    # generators
    if not DISABLE_GENLIMS:
        constr.append(cp.real(Sgen) >= gen_pmin/gen_mbase)
        constr.append(cp.real(Sgen) <= gen_pmax/gen_mbase)
        constr.append(cp.imag(Sgen) >= gen_qmin/gen_mbase)
        constr.append(cp.imag(Sgen) <= gen_qmax/gen_mbase)
    else:
        # to keep objective bounded
        constr.append(cp.real(Sgen) >= gen_pmin/gen_mbase)
    # define cost function
    quad_fac = np.diag(gencost[:,0])
    lin_fac = gencost[:,1]
    obj = cp.Minimize(cp.quad_form(cp.real(Sgen),quad_fac)+lin_fac.T@cp.real(Sgen))
    prob = cp.Problem(obj,constr)
    try:
        prob.solve(solver='GUROBI',verbose=False)
        SOLVER_ERROR = False
    except:
        print("Solver error!")
        SOLVER_ERROR = True
    data = prob.get_problem_data('GUROBI')
    print(f"Problem status is {prob.status}.")
    if prob.status not in ["infeasible", "unbounded"] and SOLVER_ERROR is False:
        # problem is feasible, display relative results
        with open(os.getcwd()+'/'+cn+'_solved.pkl','rb') as file:
            matpower_results = pickle.load(file)
        Prel = np.linalg.norm(matpower_results['gen'][:,PG][GEN_ON] - cp.real(Sgen).value*gen_mbase) / np.linalg.norm(matpower_results['gen'][:,PG][GEN_ON])
        Qrel = np.linalg.norm(matpower_results['gen'][:,QG][GEN_ON] - ((Sgen.value if Sgen.value is not None else np.zeros_like(gen_mbase))*gen_mbase)) / np.linalg.norm(matpower_results['gen'][:,QG][GEN_ON])
        # appributes
        for id,(a1,b1) in enumerate(zip(attr1,attr2)):
            ga = lambda itm,k: getattr(itm,k,None) if getattr(itm,k,None) is not None else itm
            print(f"Bus {id} P: Gen_total: {ga(a1[0],'value')}, Demand: {ga(a1[1],'value')}, Sh: {ga(a1[2],'value')}, Flows: {ga(a1[3],'value')}, baseKV: {ga(a1[4],'value')}.",flush=True)
            print(f"Bus {id} Q: Gen_total: {ga(b1[0],'value')}, Demand: {ga(b1[1],'value')}, Sh: {ga(b1[2],'value')}, Flows: {ga(b1[3],'value')}, baseKV: {ga(b1[4],'value')}",flush=True)
        # print(f"Optimal: U:{U.value},\npgen:{Pgen.value},\nqgen:{Qgen.value},\nR:{R.value},\nI:{I.value}")
        print(f"Relative ratio of optimal Pg is {Prel} and for optimal Qg is {Qrel}.\n")
        plt.plot(U.value,label='U')
        plt.plot(np.square(bus_vmin),label='vmin')
        plt.plot(np.square(bus_vmax),label="vmax")
        plt.title(cn)
        plt.legend()
        plt.show()
    else:
        # problem is infeasible
        print("Problem infeasible.\n")
    print(f"TAP:{tap},SHIFT:{shift}.")

In [ ]:
from numpy import ones, flatnonzero as find
from scipy.sparse import csr_matrix as sparse

bus, gen, baseMVA = this_case['bus'], this_case['gen'], this_case['baseMVA']
on = find(gen[:, GEN_STATUS] > 0)      ## which generators are on?
gbus = gen[on, GEN_BUS]                   ## what buses are they at?

## form net complex bus power injection vector
nb = bus.shape[0]
ngon = on.shape[0]
## connection matrix, element i, j is 1 if gen on(j) at bus i is ON
Cg = sparse((ones(ngon), (gbus, range(ngon))), (nb, ngon)).todense()

In [ ]:
Cg.shape()